In [ ]:

import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score


In [3]:
#Dam dispensing
file_name = 'Dam dispensing_header.csv'  # 원본 파일 이름
file_path = os.path.join(ROOT_DIR, file_name)
X_Dam = pd.read_csv(file_path, low_memory=True)

/var/folders/qj/004xgdtd15520sxppmd4bvvh0000gn/T/ipykernel_2970/3941712864.py:4: DtypeWarning: Columns (62,64) have mixed types. Specify dtype option on import or set low_memory=False.
  X_Dam = pd.read_csv(file_path, low_memory=True)


In [ ]:
type(X_Dam)

In [ ]:
X_Dam.info()

In [ ]:
X_Dam.describe()

In [ ]:
# pandas 출력 옵션 설정: 생략 없이 모든 열을 출력
# pd.set_option('display.max_rows', None)
X_Dam.isnull().sum()

In [4]:
# 결과를 저장할 빈 리스트 생성
results = []
# 각 열의 고유 값 개수를 저장할 빈 리스트 생성
unique_counts = []

# 각 열의 모든 값이 동일한지 확인
for column in X_Dam.columns:
    if X_Dam[column].nunique() == 1:
        print(f"All values in column '{column}' are the same.")
        results.append("same")
    else:
        print(f"Column '{column}' has different values.")
        results.append("different")
    
    unique_count = X_Dam[column].nunique()
    unique_counts.append(unique_count)

# 고유 값 개수를 데이터프레임의 마지막 행에 추가
X_Dam.loc['고유 개수'] = unique_counts
X_Dam.loc['결과'] = results
# 마지막 두 행 출력
last_two_rows = X_Dam.iloc[-2:]
print(last_two_rows)

All values in column 'Wip Line' are the same.
All values in column 'Process Desc.' are the same.
Column 'Equipment' has different values.
Column 'Model.Suffix' has different values.
Column 'Workorder' has different values.
Column 'LOT ID' has different values.
Column 'Set ID' has different values.
Column 'Box ID' has different values.
Column 'Collect Date' has different values.
Column 'Insp. Seq No.' has different values.
All values in column 'Insp Judge Code' are the same.
Column 'CURE END POSITION X_Collect Result' has different values.
Column 'CURE END POSITION X_Unit Time' has different values.
Column 'CURE END POSITION X_Judge Value' has different values.
Column 'CURE END POSITION Z_Collect Result' has different values.
Column 'CURE END POSITION Z_Unit Time' has different values.
Column 'CURE END POSITION Z_Judge Value' has different values.
Column 'CURE END POSITION Θ_Collect Result' has different values.
Column 'CURE END POSITION Θ_Unit Time' has different values.
Column 'CURE E

In [ ]:
# 마지막 행에서 'same' 값을 포함하는 열의 개수를 계산
last_row = X_Dam.iloc[-1]
same_count = (last_row == 'same').sum()

# 결과 출력
print(f"Number of columns containing 'same' in the last row: {same_count}")

In [5]:
import pandas as pd

# 마지막 행에서 'same' 값이 있는 열을 찾기
columns_to_exclude1 = X_Dam.iloc[-1][X_Dam.iloc[-1] == 'same'].index

# 마지막에서 두 번째 행에서 값이 0인 열을 찾기
columns_to_exclude2 = X_Dam.iloc[-2][X_Dam.iloc[-2] == 0].index

# 두 리스트를 합치고 중복을 제거
columns_to_exclude = list(set(columns_to_exclude1).union(set(columns_to_exclude2)))

# 해당 열을 제외한 새로운 데이터프레임 생성
X_Dam = X_Dam.drop(columns=columns_to_exclude)

print(X_Dam)

              Equipment Model.Suffix   Workorder                LOT ID  \
0      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000002   
1      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000003   
2      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000004   
3      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000005   
4      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000006   
...                 ...          ...         ...                   ...   
62476  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002687   
62477  Dam dispenser #1  AJX75334505  4F1XB738-1  OP753345054040002688   
62478  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002689   
고유 개수                 2            8         679                 59271   
결과            different    different   different             different   

                     Set ID         Collect Date Insp. Seq No.  \
0      OP753345013050000002  2023-05-04 08:57

In [6]:
# 두 열을 합치기
# NaN 값을 0으로 변환
X_Dam['WorkMode_Collect Result'] = X_Dam['WorkMode_Collect Result'].fillna(0)
X_Dam['Unnamed: 221'] = X_Dam['Unnamed: 221'].fillna(0)

X_Dam['WorkMode_Collect Result'] = X_Dam['WorkMode_Collect Result'] + X_Dam['Unnamed: 221']

# 'Unnamed: 221' 열 제거
X_Dam.drop('Unnamed: 221', axis=1, inplace=True)
print(X_Dam)

              Equipment Model.Suffix   Workorder                LOT ID  \
0      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000002   
1      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000003   
2      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000004   
3      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000005   
4      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000006   
...                 ...          ...         ...                   ...   
62476  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002687   
62477  Dam dispenser #1  AJX75334505  4F1XB738-1  OP753345054040002688   
62478  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002689   
고유 개수                 2            8         679                 59271   
결과            different    different   different             different   

                     Set ID         Collect Date Insp. Seq No.  \
0      OP753345013050000002  2023-05-04 08:57

In [ ]:
pd.set_option('display.max_rows', None)
X_Dam.isnull().sum()

In [7]:
X_Dam = X_Dam.iloc[:-2]
# 마지막 두 행 출력
last_two_rows = X_Dam.iloc[-2:]
print(last_two_rows)

              Equipment Model.Suffix   Workorder                LOT ID  \
62477  Dam dispenser #1  AJX75334505  4F1XB738-1  OP753345054040002688   
62478  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002689   

                     Set ID         Collect Date Insp. Seq No.  \
62477  OP753345054040002688  2024-04-28 18:33:51             1   
62478  OP753345054040002689  2024-04-28 18:34:57             1   

      CURE END POSITION X_Collect Result CURE END POSITION Z_Collect Result  \
62477                              240.0                                2.5   
62478                             1000.0                               12.5   

      CURE END POSITION Θ_Collect Result  ...  \
62477                                -90  ...   
62478                                 90  ...   

      Stage3 Circle3 Distance Speed_Collect Result  \
62477                                         5800   
62478                                         5800   

      Stage3 Circle4 Distance

In [ ]:

# 모델 학습 및 하이퍼파라미터 튜닝
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestClassifier(random_state=RANDOM_STATE)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='f1_weighted', cv=3, verbose=2, n_jobs=-1)
grid_search.fit(train_x, train_y)

best_model = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)


In [ ]:

# 검증 데이터로 예측 및 평가
val_pred = best_model.predict(val_x)
f1 = f1_score(val_y, val_pred, average='weighted')
print(f"F1 Score: {f1}")

# 테스트 데이터 예측
test_pred = best_model.predict(df_test_x)

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)
